# Vanilla Genetic Algorithm for Text Evolution (Baseline)

This notebook implements a standard "vanilla" GA as a baseline to compare against MATE (Memetic Algorithm for Text Evolution).

**Key Differences from MATE:**
- **No saliency-guided mutations** - uses random synonym replacement
- **No local search** - pure evolutionary operators
- **No adaptive parameters** - fixed mutation rate and static penalties
- **No gradient information** - completely stochastic

This serves to demonstrate the advantage of incorporating domain knowledge (saliency) and local refinement (memetic search) into evolutionary algorithms.

## Cell 1: Imports and Setup

In [ ]:
# Cell 1: Imports and Setup
import os
import numpy as np
import torch
from pathlib import Path
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd
import csv
import random
from nltk.corpus import wordnet
import nltk
from tqdm import tqdm

# Download WordNet if not already present
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    print("Downloading WordNet...")
    nltk.download('wordnet')
    nltk.download('omw-1.4')

# Setup directories
INPUT_FILE = Path("input.txt")
MODEL_PATH = Path("../task-2/transformer/tier_c_final_model")
GA_DIR = Path("ggs")  # Vanilla GA logs
GA_CSV = Path("vanilla_ga.csv")

# Create directories
GA_DIR.mkdir(exist_ok=True)

print("Setup complete")
print(f"  Input: {INPUT_FILE}")
print(f"  Model: {MODEL_PATH}")
print(f"  Output: {GA_DIR}")

## Cell 2: Load Classifier Model

In [ ]:
# Cell 2: Load Classifier Model
print("Loading classifier model...")

# Load PEFT configuration
peft_model_id = str(MODEL_PATH)
config = PeftConfig.from_pretrained(peft_model_id)

# Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,
    num_labels=3
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.eval()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded on {device}")
print(f"  Base: {config.base_model_name_or_path}")

## Cell 3: Load Semantic Similarity Model

In [ ]:
# Cell 3: Load Semantic Similarity Model
print("Loading sentence transformer for semantic similarity...")

semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

print("  Semantic model loaded")
print("  Model: all-MiniLM-L6-v2")

## Cell 4: Helper Functions - Classifier & Semantic Similarity

In [ ]:
# Cell 4: Helper Functions - Classifier & Semantic Similarity
def get_classifier_predictions(text):
    """
    Get classifier predictions for a text.
    Returns: (probs, predicted_class, confidence)
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
    
    predicted_class = np.argmax(probs)
    confidence = probs[predicted_class]
    
    return probs, predicted_class, confidence

def get_human_probability(text):
    """Get P(Human) = P(Class 1) for a text."""
    probs, _, _ = get_classifier_predictions(text)
    return probs[0]

def get_semantic_similarity(text1, text2):
    """
    Compute cosine similarity between two texts using sentence embeddings.
    Returns: similarity score in [0, 1]
    """
    embeddings = semantic_model.encode([text1, text2])
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    return similarity

# Test
test_text = "Technology has become an integral part of modern life."
probs, pred_class, conf = get_classifier_predictions(test_text)
print(f"Test prediction:")
print(f"  P(Human): {probs[0]:.4f}")
print(f"  P(AI): {probs[1]:.4f}")
print(f"  P(AI-mimicry): {probs[2]:.4f}")
print(f"  Predicted: Class {pred_class+1} (conf={conf:.4f})")

## Cell 5: Fitness Function with Static Penalty

In [ ]:
# Cell 5: Fitness Function with Static Penalty
def calculate_fitness(text, original_text, semantic_threshold=0.85):
    """
    Vanilla GA Fitness Function with Static Penalty.
    
    Objective: Maximize P(Human)
    Constraint: Semantic similarity must be >= 0.85
    
    If similarity < 0.85, apply a massive fixed penalty of -100.
    This is MUCH simpler than MATE's Lagrangian relaxation with adaptive weights.
    
    Args:
        text: Candidate text
        original_text: Original text for semantic comparison
        semantic_threshold: Hard threshold (0.85)
    
    Returns:
        fitness: P(Human) if constraint met, else P(Human) - 100
        components: Dict with breakdown
    """
    # Primary objective: maximize P(Human)
    p_human = get_human_probability(text)
    
    # Constraint: Semantic similarity
    similarity = get_semantic_similarity(text, original_text)
    
    # Static penalty - either you meet the constraint or you don't
    if similarity < semantic_threshold:
        penalty = 100.0  # Massive fixed penalty
    else:
        penalty = 0.0
    
    fitness = p_human - penalty
    
    components = {
        'fitness': fitness,
        'p_human': p_human,
        'similarity': similarity,
        'penalty': penalty
    }
    
    return fitness, components

# Test
original = "Technology has become an integral part of modern life."
candidate = "Tech has become a key part of everyday life."

fitness, comp = calculate_fitness(candidate, original)
print(f"Fitness test:")
print(f"  Fitness: {fitness:.4f}")
print(f"  P(Human): {comp['p_human']:.4f}")
print(f"  Similarity: {comp['similarity']:.4f}")
print(f"  Penalty: {comp['penalty']:.4f}")
print(f"\nNote: This uses a STATIC penalty unlike MATE's adaptive Lagrangian approach")

## Cell 6: Synonym-Based Text Manipulation (Random, No Saliency)

In [ ]:
# Cell 6: Synonym-Based Text Manipulation (Random, No Saliency)
def get_synonyms(word):
    """
    Get synonyms for a word using WordNet.
    Returns a list of synonyms (excluding the word itself).
    
    IMPORTANT: This is RANDOM - no saliency, no gradient information.
    """
    synonyms = set()
    
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.add(synonym)
    
    return list(synonyms)

def replace_random_words(text, replacement_rate=0.15):
    """
    Randomly replace X% of words with synonyms.
    
    CRITICAL DIFFERENCE FROM MATE:
    - This is BLIND replacement - no saliency information
    - No gradient-based selection of which words to replace
    - Just random sampling
    
    Args:
        text: Input text
        replacement_rate: Fraction of words to replace (0.15 = 15%)
    
    Returns:
        Modified text
    """
    words = text.split()
    num_to_replace = max(1, int(len(words) * replacement_rate))
    
    # Random indices to replace (no saliency guidance!)
    indices_to_replace = random.sample(range(len(words)), 
                                      min(num_to_replace, len(words)))
    
    for idx in indices_to_replace:
        word = words[idx]
        # Remove punctuation for synonym lookup
        clean_word = word.strip('.,!?;:"\'"').lower()
        
        synonyms = get_synonyms(clean_word)
        if synonyms:
            # Randomly pick a synonym (no fitness evaluation!)
            new_word = random.choice(synonyms)
            
            # Preserve capitalization
            if word[0].isupper():
                new_word = new_word.capitalize()
            
            # Preserve punctuation
            for punct in '.,!?;:\'"':
                if word.endswith(punct):
                    new_word += punct
                    break
            
            words[idx] = new_word
    
    return ' '.join(words)

# Test
test_text = "The quick brown fox jumps over the lazy dog."
print(f"Original: {test_text}")
print(f"\nRandom replacements (no saliency):")
for i in range(3):
    mutated = replace_random_words(test_text, replacement_rate=0.3)
    print(f"  {i+1}. {mutated}")

# Show synonyms for a word
print(f"\nExample synonyms for 'quick': {get_synonyms('quick')[:5]}")

## Cell 7: Initialize Population

In [ ]:
# Cell 7: Initialize Population
def initialize_population(original_text, population_size=20, mutation_rate=0.15):
    """
    Initialize population by creating random variations.
    
    DIFFERENCE FROM MATE:
    - MATE uses Gemini to generate semantically diverse variations
    - Vanilla GA uses simple random synonym replacement
    - Much less diverse, more likely to get stuck in local optima
    
    Args:
        original_text: Source text
        population_size: Number of individuals (20 for vanilla GA)
        mutation_rate: Fraction of words to randomly mutate (0.15)
    
    Returns:
        population: List of text individuals
    """
    print(f"Initializing population of {population_size} individuals...")
    
    population = []
    
    # First individual: original text
    population.append(original_text)
    print(f"  [1/{population_size}] Original text")
    
    # Generate variations via random synonym replacement
    for i in range(2, population_size + 1):
        variant = replace_random_words(original_text, replacement_rate=mutation_rate)
        population.append(variant)
        print(f"  [{i}/{population_size}] Random variant")
    
    print(f"Population initialized: {len(population)} individuals")
    return population

print("Population initialization function defined")

## Cell 8: Selection Operator - Tournament Selection

In [ ]:
# Cell 8: Selection Operator - Tournament Selection
def tournament_selection(population, fitnesses, tournament_size=3):
    """
    Tournament Selection: Pick k random individuals, return the best one.
    
    This is standard GA selection - no fancy ranking or adaptive pressure.
    
    Args:
        population: List of individuals
        fitnesses: List of fitness scores
        tournament_size: Number of individuals in tournament (k=3)
    
    Returns:
        Selected individual
    """
    # Randomly pick k individuals
    tournament_indices = random.sample(range(len(population)), tournament_size)
    
    # Find the best one
    best_idx = tournament_indices[0]
    best_fitness = fitnesses[best_idx]
    
    for idx in tournament_indices[1:]:
        if fitnesses[idx] > best_fitness:
            best_idx = idx
            best_fitness = fitnesses[idx]
    
    return population[best_idx]

print("Tournament selection defined (k=3)")

## Cell 9: Crossover Operator - One-Point Crossover

In [ ]:
# Cell 9: Crossover Operator - One-Point Crossover
def one_point_crossover(parent1, parent2):
    """
    One-Point Crossover for text.
    
    Split both parents at a random word position.
    Combine Head(Parent1) + Tail(Parent2).
    
    DIFFERENCE FROM MATE:
    - MATE uses Gemini to semantically blend styles
    - Vanilla GA uses mechanical word-level concatenation
    - Can create awkward, ungrammatical offspring
    
    Args:
        parent1: First parent text
        parent2: Second parent text
    
    Returns:
        child: Offspring text
    """
    words1 = parent1.split()
    words2 = parent2.split()
    
    # Avoid splitting at the very beginning or end
    if len(words1) < 3 or len(words2) < 3:
        return parent1  # Too short for meaningful crossover
    
    # Random crossover point
    crossover_point = random.randint(1, len(words1) - 1)
    
    # Create child: first part from parent1, second part from parent2
    # Adjust second parent's split to maintain reasonable length
    second_part_start = random.randint(0, max(0, len(words2) - (len(words1) - crossover_point)))
    
    child_words = words1[:crossover_point] + words2[second_part_start:]
    child = ' '.join(child_words)
    
    return child

# Test
parent1 = "The quick brown fox jumps over the lazy dog every morning."
parent2 = "A fast red cat leaps across the sleeping hound each evening."

print("Crossover test:")
print(f"  Parent 1: {parent1}")
print(f"  Parent 2: {parent2}")
print(f"\nOffspring examples:")
for i in range(3):
    child = one_point_crossover(parent1, parent2)
    print(f"  {i+1}. {child}")

## Cell 10: Mutation Operator - Uniform Random Mutation

In [ ]:
# Cell 10: Mutation Operator - Uniform Random Mutation
def uniform_mutation(text, mutation_rate=0.1):
    """
    Uniform Random Mutation.
    
    For each word, with probability mutation_rate:
    - Replace it with a random synonym
    
    CRITICAL DIFFERENCE FROM MATE:
    - MATE uses saliency to identify WHICH words to mutate
    - MATE uses Gemini to generate contextually appropriate replacements
    - Vanilla GA blindly mutates random words
    - No gradient information, no intelligence
    
    This is the KEY weakness of vanilla GA for this problem.
    
    Args:
        text: Input text
        mutation_rate: Probability of mutating each word (0.1 = 10%)
    
    Returns:
        Mutated text
    """
    words = text.split()
    
    for i in range(len(words)):
        if random.random() < mutation_rate:
            word = words[i]
            # Remove punctuation for synonym lookup
            clean_word = word.strip('.,!?;:"\'"').lower()
            
            synonyms = get_synonyms(clean_word)
            if synonyms:
                # Randomly pick a synonym (no fitness check!)
                new_word = random.choice(synonyms)
                
                # Preserve capitalization
                if word and word[0].isupper():
                    new_word = new_word.capitalize()
                
                # Preserve punctuation
                for punct in '.,!?;:\'"':
                    if word.endswith(punct):
                        new_word += punct
                        break
                
                words[i] = new_word
    
    return ' '.join(words)

# Test
test_text = "The quick brown fox jumps over the lazy dog every morning."
print("Mutation test (mutation_rate=0.3 for visibility):")
print(f"  Original: {test_text}")
print(f"\nRandom mutations:")
for i in range(3):
    mutated = uniform_mutation(test_text, mutation_rate=0.3)
    print(f"  {i+1}. {mutated}")

## Cell 11: Logging Functions

In [ ]:
# Cell 11: Logging Functions
def initialize_csv():
    """Initialize CSV file with headers."""
    with open(GA_CSV, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            'generation', 'individual_id', 'fitness', 'p_human', 'p_ai', 'p_mimicry',
            'semantic_similarity', 'penalty', 'predicted_class', 'is_best'
        ])

def log_to_csv(generation, population, fitnesses, original_text):
    """Log all individuals in a generation to CSV."""
    best_idx = np.argmax(fitnesses)
    with open(GA_CSV, "a", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        for i, (text, fitness) in enumerate(zip(population, fitnesses)):
            _, components = calculate_fitness(text, original_text)
            probs, pred_class, _ = get_classifier_predictions(text)
            writer.writerow([
                generation, i+1, f"{fitness:.8f}",
                f"{probs[0]:.8f}", f"{probs[1]:.8f}", f"{probs[2]:.8f}",
                f"{components['similarity']:.4f}",
                f"{components['penalty']:.4f}",
                pred_class+1, 1 if i==best_idx else 0
            ])

def log_generation(generation, population, fitnesses, original_text):
    """Log generation details to text file."""
    filepath = GA_DIR / f"generation_{generation}.txt"
    
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(f"VANILLA GA - GENERATION {generation}\n")
        f.write("=" * 80 + "\n\n")
        
        # Sort by fitness
        sorted_indices = np.argsort(fitnesses)[::-1]
        
        for rank, idx in enumerate(sorted_indices[:5], 1):  # Top 5 only
            text = population[idx]
            fitness = fitnesses[idx]
            
            _, components = calculate_fitness(text, original_text)
            probs, pred_class, conf = get_classifier_predictions(text)
            
            f.write(f"Rank {rank} | Individual {idx + 1}\n\n")
            f.write(f"Fitness: {fitness:.4f}\n")
            f.write(f"  - P(Human): {components['p_human']:.4f}\n")
            f.write(f"  - P(AI): {probs[1]:.4f}\n")
            f.write(f"  - P(AI-mimicry): {probs[2]:.4f}\n")
            f.write(f"  - Semantic similarity: {components['similarity']:.4f}\n")
            f.write(f"  - Penalty: {components['penalty']:.4f}\n")
            f.write(f"  - Predicted class: {pred_class+1} (conf={conf:.4f})\n")
            f.write(f"\nText:\n{text}\n")
            f.write(f"\n{'=' * 80}\n\n")
        
        # Summary
        f.write(f"SUMMARY STATISTICS\n")
        f.write(f"Best fitness: {max(fitnesses):.4f}\n")
        f.write(f"Avg fitness:  {np.mean(fitnesses):.4f}\n")
        f.write(f"Worst fitness: {min(fitnesses):.4f}\n")

print("Logging functions defined")